# 01 — Télécharger & préparer le dataset (classe **frelon** uniquement)

Ce notebook :
1. Télécharge le dataset Roboflow depuis votre compte (vous devez fournir votre `RF_API_KEY`).
2. Extrait uniquement la classe **frelon**.
3. Construit des annotations *COCO* filtrées (1 seule catégorie) et génère des TFRecords pour la TensorFlow Object Detection API.

## 1) Paramètres

In [ ]:
# ⚠️ À adapter si besoin
ROBOFLOW_API_KEY = os.environ.get("RF_API_KEY", "")  # ou mettez directement votre clé: "rf_xxx"
ROBOFLOW_WORKSPACE = "frelonproject"
ROBOFLOW_PROJECT = "hornet-bees-6bl4k"
ROBOFLOW_VERSION = 2  # selon votre lien
DATA_BASE_DIR = "data"  # dossier local où placer les données

# Nom de la classe à garder
TARGET_CLASS = "frelon"

os.makedirs(DATA_BASE_DIR, exist_ok=True)
print("DATA_BASE_DIR:", os.path.abspath(DATA_BASE_DIR))

## 2) Télécharger en format COCO (object detection)

In [ ]:
from roboflow import Roboflow
assert ROBOFLOW_API_KEY, "Veuillez renseigner votre clé Roboflow (RF_API_KEY)."

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(ROBOFLOW_VERSION)

# Télécharge en COCO pour TF (images + annotations)
dataset = version.download("coco")

print("Dataset local:", dataset.location)

## 3) Filtrer les annotations à la seule classe **frelon**

In [ ]:
import json, shutil, glob, os

coco_dirs = {
    "train": os.path.join(dataset.location, "train"),
    "valid": os.path.join(dataset.location, "valid"),
    "test":  os.path.join(dataset.location, "test"),
}

def filter_coco_to_single_class(coco_json_path, keep_class_name, out_json_path):
    with open(coco_json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    # Trouver l'id de la classe à garder
    name_to_id = {c["name"]: c["id"] for c in coco["categories"]}
    if keep_class_name not in name_to_id:
        raise ValueError(f"Classe '{keep_class_name}' introuvable dans {coco_json_path}.")

    keep_id = name_to_id[keep_class_name]

    # Filtrer annotations et images
    keep_ann = [a for a in coco["annotations"] if a["category_id"] == keep_id]
    keep_img_ids = set(a["image_id"] for a in keep_ann)
    keep_imgs   = [img for img in coco["images"] if img["id"] in keep_img_ids]

    # Nouvelle catégorie unique
    new_cats = [{
        "id": 1,
        "name": keep_class_name,
        "supercategory": keep_class_name
    }]

    # Remapper category_id -> 1
    for a in keep_ann:
        a["category_id"] = 1

    filtered = {
        "images": keep_imgs,
        "annotations": keep_ann,
        "categories": new_cats
    }

    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(filtered, f, ensure_ascii=False)
    return len(keep_imgs), len(keep_ann)

for split in ["train", "valid", "test"]:
    ann_path = os.path.join(coco_dirs[split], "_annotations.coco.json")
    out_path = os.path.join(coco_dirs[split], f"_annotations_single_{TARGET_CLASS}.coco.json")
    n_img, n_ann = filter_coco_to_single_class(ann_path, TARGET_CLASS, out_path)
    print(f"{split}: images gardées={n_img}, annotations gardées={n_ann}")

## 4) Générer des TFRecords (TF Object Detection API)

In [ ]:
# Utilise le script officiel create_coco_tf_record.py du repo TF Models.
# Le script sera disponible après installation de l'API (voir notebook 02).
# Ici, on prépare simplement les chemins et les commandes à exécuter plus tard.

TF_MODELS_DIR = "models"  # où sera cloné le repo tensorflow/models
TF_RECORDS_DIR = os.path.join(DATA_BASE_DIR, "tfrecords_single_class")
os.makedirs(TF_RECORDS_DIR, exist_ok=True)

train_images_dir = os.path.join(coco_dirs["train"])
val_images_dir   = os.path.join(coco_dirs["valid"])
test_images_dir  = os.path.join(coco_dirs["test"])

train_ann = os.path.join(coco_dirs["train"], f"_annotations_single_{TARGET_CLASS}.coco.json")
val_ann   = os.path.join(coco_dirs["valid"], f"_annotations_single_{TARGET_CLASS}.coco.json")

print("TF_RECORDS_DIR:", os.path.abspath(TF_RECORDS_DIR))

print("""
Après avoir installé l'API (notebook 02), exécutez **dans un terminal** depuis la racine du repo 'models':

python research/object_detection/dataset_tools/create_coco_tf_record.py \
  --logtostderr \
  --train_image_dir {train_images_dir} \
  --val_image_dir {val_images_dir} \
  --test_image_dir {test_images_dir} \
  --train_annotations_file {train_ann} \
  --val_annotations_file {val_ann} \
  --testdev_annotations_file {val_ann} \
  --output_dir {TF_RECORDS_DIR}
""")